
### Setup

In this section, we set up our environment by importing necessary libraries and configurations. We also suppress specific warnings to ensure a clean output.


In [ ]:
import warnings
import pandas as pd 
warnings.filterwarnings('ignore')


### Scraping Reviews from Rotten Tomatoes

To analyze the writing style of different critics, we start by scraping reviews from Rotten Tomatoes for a list of specified critics. This data will serve as our base for further analysis and interaction with the language model.


In [ ]:
CRITICS = {
    "REUBEN BARON": "https://www.rottentomatoes.com/critics/reuben-baron/movies",
    "RON SEOUL-OH": "https://www.rottentomatoes.com/critics/ron-seoul-oh/movies",
    "LINDA MARRIC": "https://www.rottentomatoes.com/critics/linda-marric/movies",
    "ISHITA SENGUPTA": "https://www.rottentomatoes.com/critics/ishita-sengupta/movies",
    "GANESH AAGLAVE": "https://www.rottentomatoes.com/critics/ganesh-aaglave/movies",
    "DANA KENNEDY": "https://www.rottentomatoes.com/critics/dana-kennedy/movies",
    "NATALIA KEOGAN": "https://www.rottentomatoes.com/critics/natalia-keogan/movies"
}

In [ ]:
data = pd.DataFrame()
for critic, url in CRITICS.items():
    tmp_df = pd.read_html(url)[0]
    tmp_df['Critic'] = critic
    data = data.append(tmp_df)

In [ ]:
data.head()


### Data Cleaning and Transformation

After scraping the reviews, we need to clean and transform the data to make it suitable for analysis. The first step is to normalize the ratings.


In [ ]:
def normalize_rating(rating):
    """
    Normalize a rating represented as a string fraction or a zero value.
    
    Examples:
        >>> normalize_rating('1/5')
        20.0
        >>> normalize_rating('2/0')
        None
        >>> normalize_rating(0)
        0.0
        >>> normalize_rating('invalid_string')
        None
    """
    if isinstance(rating, str) and '/' in rating:
        try:
            numerator, denominator = map(float, rating.split('/'))
            if denominator != 0:  # Avoid division by zero
                return (numerator / denominator) * 100
        except ValueError:  # Handle cases where the string isn't two numbers separated by '/'
            return None
    elif rating == 0:
        return 0.0
    return None

In [ ]:
data['Rating'] = data.Rating.apply(normalize_rating)
data = data.dropna(subset=['Rating'])


Next, we categorize the normalized ratings into sentiment bins, extract movie titles and years, and count the number of reviews for each critic. We also remove any unnecessary columns.


In [ ]:
# Define bins and labels for sentiment categorization
bins = [-1, 50, 70, 90, 100]  # We use -1 as the start to include 0 in the first bin
labels = ['bad', 'neutral', 'positive', 'very positive']

data['Sentiment'] = pd.cut(data['Rating'], bins=bins, labels=labels, right=True)
data[['Title', 'Year']] = data['Title | Year'].str.extract(r'(.+)\s\((\d{4})\)')
data['critic_count'] = data.groupby('Critic')['Critic'].transform('count')
data.drop(columns=['Title | Year', 'T-Meter'], inplace=True)

### Data Augmentation

First, we filter out critics with fewer reviews.

In [ ]:
# Filter data to keep critics with a minimum number of reviews
MIN_REVIEWS = 5
filter_mask = (data.critic_count >= MIN_REVIEWS)
data = data[filter_mask]

Next, we augment each row with additional context. This helps in better understanding the sentiment and rating associated with each review.

In [ ]:
def augment_context(row):
    return f"""
    CRITIC: {row['Critic']}
    RATING: {row['Rating']}
    SENTIMENT: {row['Sentiment']}
    MOVIE: {row['Title']}
    YEAR: {row['Year']}
    REVIEW: {row['Review']}
    """

In [ ]:
data['Context'] = data.apply(augment_context, axis=1)

In [ ]:
data.head()


### Vector Store Configuration

To enable semantic search over our dataset, we utilize a vector store. This allows us to efficiently find reviews that are semantically similar to a given query.



We'll generate embeddings for each review. These embeddings are dense vector representations that capture the semantic meaning of the text. They will be used for similarity searches in the vector store.


In [ ]:
from tqdm import tqdm
tqdm.pandas()
from config import get_embedding, get_chat_completion

data["embeddings"] = data["Context"].progress_apply(get_embedding)
data.reset_index(inplace=True, drop=True)


With the embeddings generated, we can now configure and set up the vector store. This will allow us to efficiently query reviews based on semantic similarity.


In [ ]:
import chromadb

client = chromadb.PersistentClient()
collection = client.get_or_create_collection("rotten_tomatoes_reviews")

In [ ]:
embeddings = data.embeddings.values.tolist()
ids = data.index.astype(str).values.tolist()
documents = data.Context.values.tolist()
metadata = data[['Rating', 'Review', 'Critic', 'Sentiment', 'Title', 'Year', 'critic_count', 'Context']].to_dict(orient='records')

In [ ]:
collection.add(
    embeddings=embeddings,
    metadatas=metadata,
    documents=documents,
    ids=ids,
)


### Sample Run

In this demonstration, we illustrate the end-to-end process of crafting a review in the style of a chosen critic using the LLM. We utilize the vector store to fetch reviews that match the critic's style and construct a few-shot prompt to guide the LLM's response.



#### Selection

First, we choose a critic, sentiment, movie, and draft the initial review details.


In [ ]:
data.Critic.unique()

In [ ]:
CRITIC = "NATALIA KEOGAN"
SENTIMENT = "positive"
MOVIE = "Whispers of the Nebula"
MOVIE_DESCRIPTION = """In a distant future, a renowned astronaut discovers an ancient alien civilization within a mesmerizing nebula, only to realize that the echoes of their history hold the key to saving Earth from an impending cosmic disaster."""
REVIEW_ROUGH_DRAFT = """
- Cool space adventures meet old alien secrets.
- Makes you think about Earth's place in space.
- Mixes space tales with ancient puzzles.
"""
INITIAL_PROMPT = f"""
Write a {SENTIMENT} review about a {MOVIE}. I want you to write the review in the style of {CRITIC}.

The movie description is: {MOVIE_DESCRIPTION}

Here are things I want to hit on in my review: {REVIEW_ROUGH_DRAFT}
"""


#### Few-Shot Prompting

Constructing the few-shot prompt by incorporating the retrieved reviews from the vector store.


In [ ]:
prompt_template = """
HERE IS MY TASK: {prompt}

==== EXAMPLE OUTPUTS ====
{few_shot_examples}
===== END OF EXAMPLE OUTPUTS =====

Mirror the example outputs given and return one response. Think step-by-step about how you would respond to the prompt. OUTPUT:
"""


#### Semantic Search

We use the vector store to retrieve top reviews that match the style of the chosen critic.


In [ ]:
TOP_K = 10
prompt_embedding = get_embedding(INITIAL_PROMPT)
query_result = collection.query(prompt_embedding, n_results=TOP_K, include=["documents", "distances", "metadatas"], where={"Critic": CRITIC})

few_shot_examples = """ """
for i, result in enumerate(query_result["metadatas"][0]):
    review = f"""
    Example Output {i + 1}: {result["Review"]}
    """
    few_shot_examples += review

prompt = prompt_template.format(prompt=INITIAL_PROMPT, few_shot_examples=few_shot_examples)
print(prompt)

#### Interacting with the LLM
We send the few-shot prompt to the LLM to get a review for the chosen movie in the style of the selected critic.

Zero-Shot Example

In [47]:
messages = [{"role": "user", "content": INITIAL_PROMPT}]
zero_shot_output = get_chat_completion(messages, model="gpt-4")
print(zero_shot_output)

**Few-Shot Example**

In [ ]:
messages = [{"role": "user", "content": prompt}]
few_shot_output = get_chat_completion(messages, model="gpt-4")
print(few_shot_output)